# Jenkins 101
------
<div class="logo" style="padding:0; display:flex; align-items:center">
    <img alt="Jenkins" src="jenkins.svg" style="margin: 0; margin-right: 20px">
    <div style="font-size:144px;font-family:'Liberation Serif';font-weight:bold">Jenkins</div>
</div>

### Kevin Howell
#### 2019-06-16 - Southeast Linux Fest

# Jenkins 101 - Goals
------

## Gain a basic understanding of Jenkins

## Learn how to install & manage Jenkins

## Learn some Jenkins best practices

# Jenkins 101 - About Me
------
### khowell@redhat.com (or kevin@kahowell.net)
### https://github.com/kahowell
### https://kahowell.net
### Red Hat Certified Architect
### Senior Software Engineer & tech lead for Candlepin & subscription-manager team at Red Hat (Satellite 6)

<div>
    <img src="Red_Hat_Logo_2019.svg" style="width: 20%">
    <img src="candlepin_logo.png" style="width:10%">
</div>

# Jenkins 101 - What is Jenkins?
------

### Jenkins is a widely used open source automation server

- If anyone has used Hudson, that's the project Jenkins forked from

### Indicated, common use is to use Jenkins to build and test software

### Jenkins can be used as a general purpose tool for centrally executing tasks

- A centrally accessible web interface for triggering and monitoring automation

### Jenkins functionality is provided by a community of plugins

- There is a vast ecosystem of plugins; they often make functionality more "native" to Jenkins; usually you can use shell scripts when a plugin doesn't exist or if you can't install Jenkins plugins due to security constraints or lack of access.

# Jenkins 101 - The Server
------
<div style="float:right; width: 25%">
    <img src="logo-rhel8.svg" style="width: 100%">
    <hr>
    <img src="Windows_darkblue_2012_svg.svg" style="width:55%">
    <hr>
    <img src="MacOS_wordmark_(2017).svg" style="width:30%">
</div>

The Jenkins server can run anywhere that Java runs.

Jenkins creates packages for many platforms:

- Linux
 - RHEL, Centos, Fedora
 - OpenSUSE
 - Ubuntu, Debian
 - Container Image
- Windows
- macOS

# Jenkins 101 - Installation
------

### Use your favorite package manager (https://jenkins.io has packages for nearly everything).

### Run a VM image

- Preconfigured VM image from your cloud provider of choice
- Custom VM image to be launched in your cloud provider of choice using https://packer.io or other similar tools.

### Run a container image

- docker.io/library/jenkins
- OpenShift has a Jenkins image

# Jenkins 101 - Server Interface
------

The primary way to interact with the server is through its Web interface, e.g. http://localhost:8080.

<img src="jenkins_ui.png">

- show plugins pages
- show security
- show credentials

# Jenkins 101 - Jenkins Nodes
------
It's **best practice** in Jenkins to use systems other than the server in order to execute tasks.

### Security

- Create nodes for specific tasks, and authorize those nodes to perform specific tasks
- Example, you might put an ssh key on the node that the Jenkins user is able to use (compare to centrally storing on Jenkins).
- A compromised node doesn't need to mean a compromised Jenkins server.

### Stability

- Don't tax the Jenkins server/master itself.
- Scale up and down compute resources as necessary.

### Isolation

- provide a consistent level of service for each group/project/org using Jenkins.

### Cross-platform automation

- build or test your application against multiple platforms
- with subscription-manager we test functionality against SUSE based systems using this.

# Jenkins 101 - Types of Jenkins Nodes
------
There are many ways to use systems as Jenkins nodes:

### The Jenkins server itself is a Jenkins node
### SSH to system with Java installed

- show ssh agent add here

# Jenkins 101 - Types of Jenkins Nodes
------
### Provision and deprovision an on-premise VM on-demand (e.g. via OpenStack)
<div style="float:right; width:25%">
    <img src="OpenStack_Logo2016.svg">
</div>

# Jenkins 101 - Types of Jenkins Nodes
------
### Provision and deprovision a cloud-provider VM on-demand (e.g. via Amazon EC2)
<div style="float:right; width:25%">
    <img src="Amazon_Web_Services_Logo.svg" style="width:50%"><img style="width:140%" src="Google_Cloud_Logo.svg"><img src="Microsoft_Azure_Logo.svg">
</div>

# Jenkins 101 - Types of Jenkins Nodes
------
### Provision and deprovision containers or pods on a specific system on-demand
<div style="float:right; width:25%">
    <img src="kubernetes-horizontal-color.svg">
</div>

# Jenkins 101 - Freestyle Jobs
------
The original method of defining execution via Jenkins is the Job.

Jobs are broken up into several components:

### Job configuration (e.g. git configuration, parameters)

- retention
- parameters
- scm information
- description is useful for documenting the job

### Triggers

- trigger on git push
- scheduled job w/ cron-like schedule

### Build Actions

- think shell scripts as a baseline
- can use parameters
- can use credentials

### Post-Build Actions (e.g. send email)

- convenience methods for notification mainly
- chain jobs

# Jenkins 101 - Freestyle Job Example - Web Interface
------
<img src="freestyle_job_ui.png">

# Jenkins 101 - Freestyle Job Example - Job DSL
------
Snippet from https://github.com/candlepin/candlepin-jobs/blob/8431af34901ee547f0d79474665dc76e28e7a308/src/jobs/submanVagrantUpstreamImagesJob.groovy:

```groovy
job("$baseFolder/vagrant-upstream-images") {
    description('builds centos, fedora, etc. vagrant images for subman development')
    label('rhsm-packer')
    scm {
        github('candlepin/packer', 'master')
    }
    wrappers {
        colorizeOutput()
        credentialsBinding {
            string('VAGRANT_CLOUD_TOKEN', 'VAGRANT_CLOUD_TOKEN')
        }
    }
    steps {
        shell(readFileFromWorkspace('src/resources/subman-vagrant-images.sh'))
    }
}
```

# Jenkins 101 - Pipelines
------
The preferred, modern way of automating with Jenkins is by implementing Jenkins pipelines.

<img src="blue_ocean.png">

It was once common practice to create software pipelines by chaining Jenkins jobs together, but now pipelines are a first-class concept in Jenkins.

# Jenkins 101 - Example Pipeline
------
```groovy
pipeline {
    options { buildDiscarder(logRotator(numToKeepStr: '50')) }
    agent {
        label 'rhsm'
    }
    stages {
        stage('Clean') {
            steps {
                sh './gradlew --no-daemon clean'
            }
        }
        stage('Build') {
            steps {
                sh './gradlew --no-daemon assemble'
            }
        }
        /* more stages omitted */
    }

    post {
        always {
            archiveArtifacts artifacts: 'build/reports/checkstyle/*.html'
            junit 'build/test-results/**/*.xml'
        }
    }
}
```

# Jenkins 101 - Pipeline Advice
------

### `Jenkinsfile`, optional, but recommended

- Think Makefile or .travis.yml

### Use the "declarative" syntax when possible

- Declarative syntax provides more constraints, leads to less hacky code
- can always use `script` blocks to break out into DSL

### Use script files (.sh, .py)

- lots of people already know bash scripting or python, ruby, etc.
- can use the scripts outside of Jenkins

# Jenkins 101 - Jenkins as a Central Script Runner
------

### Centralized way to run scripts

- share your scripts
- please keep them in a git repo

### Empower people to use root powers, without giving *them* root.

- give Jenkins and/or Jenkins nodes access and then control access to the job/pipeline
- think who avoid scripting for fear, inexperience, or distrust

### Share output with colleagues easily

- Jenkins keeps logs

### Keep a history of who and what

- keep retention and storage in mind

# Jenkins 101 - Jenkins for Continous Integration
------
Use Jenkins to test incoming changes:

### Unit tests

- maintained in the code

### End-to-end tests

- stand up your application
- use same repo or multiple repos

### Static analysis

- flake8
- findbugs
- sonarqube

### PRs or long-lived branches (e.g. master)

- PRs for testing incoming changes
- test long-lived branches to keep dev & hotfix branches stable, and keep a baseline

# Jenkins 101 - Jenkins for Continous Delivery
------
Use Jenkins to automate building and releasing your products:

### Automate builds on various platforms

- example, if you deliver an RPM and a Windows EXE installer

### Automate delivery to a CDN

- Give Jenkins access to push your built releases to a CDN

### Automate deployment of a service

- Give Jenkins access to your platform of choice
- Use in conjuction with puppet, ansible, etc.

# Jenkins 101 - Recap
------
- Jenkins can be used for lots of kinds of automation
- Plugins can be installed to provide additional functionality
- Jenkins nodes allow you automate various platforms and isolate your automation
- Pipelines should be used for new automation

# Jenkins 101 - Resources
------
- Jenkins: https://jenkins.io
- Jenkins Job DSL: https://github.com/jenkinsci/job-dsl-plugin
- Job DSL Examples: https://github.com/sheehan/job-dsl-gradle-example
 - Real-World Jenkins Job Examples: https://github.com/candlepin/candlepin-jobs
- Jenkins Pipeline Reference: https://jenkins.io/doc/pipeline/steps/
- Jenkins Pipeline Examples: https://github.com/jenkinsci/pipeline-examples
 - Real-World Jenkins Pipeline Example: https://github.com/RedHatInsights/rhsm-conduit/blob/master/Jenkinsfile

# Jenkins 101 - Q&A & Thank You!
------
Thanks for attending! Questions?

<div style="display: flex">
    <div style="width: 50%;">
        <div style="float: left">
            <img src="presentation_link.svg" style="width: 400px">
            <h3>Presentation: <a href="http://bit.ly/2KNvyIE">http://bit.ly/2KNvyIE</a></h3>
        </div>
    </div>
    <div style="width: 50%">
        <div style="float: right">
            <img src="survey.svg" style="width: 400px">
            <h3>Feedback: <a href="https://forms.gle/Jgh1dPmMYTbq17KWA">https://forms.gle/Jgh1dPmMYTbq17KWA</a></h3>
        </div>
    </div>
</div>